In [ ]:
import tweepy
import pandas as pd
import time
import requests
import langtags

In [ ]:
client = tweepy.Client(bearer_token="bearer_token",
                       access_token="access_token",
                       access_token_secret="access_token_secret")

### Query for tweets containing the URL "opencollective.com/" and "github.com"

In [ ]:
tweet_data = []
for tweet in tweepy.Paginator(
        client.search_all_tweets,
        query=
        'url:"opencollective.com/" url:"github.com" -is:retweet -is:reply',
        tweet_fields=[
            'id', 'text', 'author_id', 'created_at', 'entities',
            'referenced_tweets', 'lang', 'public_metrics'
        ],
        user_fields=['username'],
        expansions='author_id',
        start_time='2019-05-01T00:00:00Z',
        end_time='2022-04-30T00:00:00Z',
        max_results=50):
    time.sleep(2)
    tweet_data.append(tweet)

### Extract the data from the obtained

In [ ]:
data = []
urls = []
oc_user = []
github_user = []
get_username = True
user_dict = {}
headers = {
    'user-agent':
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36'
}
# Loop through each response object
for response in tweet_data:
    if str(response.data) != "None":
        # Take all of the users, and put them into a dictionary of dictionaries with the info we want to keep
        for user in response.includes['users']:
            user_dict[user.id] = {'username': user.username}
        for tweet in response.data:
            # For each tweet, find the author's information
            author_info = user_dict[tweet.author_id]
            # Get links for each tweet
            for i in range(len(tweet.entities['urls'])):
                link = tweet.entities['urls'][i]['expanded_url']
                urls.append(link)
                if 'opencollective.com/' in link:
                    oc_user.append(link.split('/')[3].split('?')[0])
                    get_username = False
                elif 'github.com/sponsors/' in link:
                    github_user.append(
                        link.split('/')[4].split('?')[0].split('#')[0])
                    get_username = False
                elif 'gist.github.com/' in link:
                    github_user.append(link.split('/')[3].split('?')[0])
                    get_username = False
                elif 'github.com/' in link:
                    github_user.append(link.split('/')[3].split('?')[0])
                    get_username = False
                # If the URL has been shortened, it should be extended
                elif get_username:
                    try:
                        short_link = requests.head(link,
                                                   allow_redirects=True,
                                                   headers=headers)
                    except requests.exceptions.RequestException as error:
                        print("Error: ", error)
                    if 'opencollective.com/' in str(short_link.url):
                        oc_user.append(
                            str(short_link.url).split('/')[3].split('?')[0])
                    elif 'github.com/sponsors/' in str(short_link.url):
                        github_user.append(
                            str(short_link.url).split('/')[4].split('?')
                            [0].split('#')[0])
                    elif 'gist.github.com/' in str(short_link.url):
                        github_user.append(
                            str(short_link.url).split('/')[3].split('?')[0])
                    elif 'github.com/' in str(short_link.url):
                        github_user.append(
                            str(short_link.url).split('/')[3].split('?')[0])
                get_username = True
                time.sleep(1.5)
            data.append({
                'Tweet_Url': str(tweet.id),
                'Language': tweet.lang,
                'Time': tweet.created_at,
                'Tweet_User_ID': tweet.author_id,
                'Tweet_Username': author_info['username'],
                'Tweet_Text': tweet.text,
                'Opencollective_Username': oc_user,
                'Sponsor_Username': github_user,
                '# Retweet': tweet.public_metrics['retweet_count'],
                '# Replys': tweet.public_metrics['reply_count'],
                '# Likes': tweet.public_metrics['like_count'],
                'Links': urls
            })
            urls = []
            github_user = []
            oc_user = []

### Convert to pandas format

In [ ]:
df = pd.DataFrame(data)

### Convert to tweet url

In [ ]:
df['Tweet_Url'] = 'https://twitter.com/twitter/status/' + df[
    'Tweet_Url'].astype(str)

### Decode the language

In [ ]:
lang = []
for a in df['Language']:
    lang.append(langtags.Tag(a).language.description)
df['Language'] = lang

### Save as csv file

In [ ]:
df.to_csv('../../data/Contribution_activities/OpenCollective_tweets.csv')